In [ ]:
import requests
import json
import os

# --- IMPORTANT SECURITY NOTE ---
# This script is designed to use Google Colab's "Secrets" feature
# to securely store your API keys.
# 1. In your Colab notebook, click the "Secrets" icon (key icon 🔑) in the left sidebar.
# 2. Add two new secrets:
#    - Name: GEMINI_API_KEY, Value: Your Google Gemini API Key
#    - Name: OPENAI_API_KEY, Value: Your OpenAI (ChatGPT) API Key
# 3. Ensure "Notebook access" is toggled ON for both secrets.
# Never hardcode your API keys directly in the script for production use.

print("--- AI-to-AI Interaction Script (Gemini to ChatGPT) ---")
print("This script allows Gemini to formulate a question for ChatGPT,")
print("send it to ChatGPT, and then analyze ChatGPT's response.")
print("\n")

# --- 1. Get API Keys from Colab Secrets ---
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

    if not GEMINI_API_KEY:
        raise ValueError("GEMINI_API_KEY not found in Colab Secrets. Please add it.")
    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY not found in Colab Secrets. Please add it.")

except ImportError:
    print("Warning: 'google.colab.userdata' not found. This script is intended for Google Colab.")
    print("Falling back to environment variables or manual input (less secure).")
    GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')
    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

    if not GEMINI_API_KEY:
        GEMINI_API_KEY = input("Please enter your Google Gemini API Key: ").strip()
    if not OPENAI_API_KEY:
        OPENAI_API_KEY = input("Please enter your OpenAI (ChatGPT) API Key: ").strip()

    if not GEMINI_API_KEY or not OPENAI_API_KEY:
        print("Error: API Keys are missing. Please provide them.")
        exit()
except ValueError as e:
    print(f"Error: {e}")
    print("Please ensure your API keys are correctly set up in Colab Secrets.")
    exit()


# --- 2. Function to Interact with Gemini API ---
def ask_gemini(prompt, model_name="gemini-2.0-flash", api_key=GEMINI_API_KEY):
    """Sends a prompt to the Gemini API and returns the generated text."""
    url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent?key={api_key}"
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt}
                ]
            }
        ]
    }

    print(f"\n--- Asking Gemini ({model_name})... ---")
    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        response_data = response.json()

        if response_data.get("candidates") and response_data["candidates"][0].get("content"):
            gemini_text = response_data["candidates"][0]["content"]["parts"][0]["text"]
            print(f"Gemini's Response:\n{gemini_text}")
            return gemini_text
        else:
            print("Gemini: No content received or unexpected response structure.")
            # print(f"Full Gemini Response: {response_data}") # Uncomment for detailed debugging
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error communicating with Gemini API: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON from Gemini API. Response text: {response.text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred with Gemini: {e}")
        return None

# --- 3. Function to Interact with ChatGPT API ---
def ask_chatgpt(question, model_name="gpt-4o", api_key=OPENAI_API_KEY):
    """Sends a question to the ChatGPT API and returns its response."""
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": question}
        ]
    }

    print(f"\n--- Asking ChatGPT ({model_name})... ---")
    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        response_data = response.json()

        if response_data.get("choices") and response_data["choices"][0].get("message"):
            chatgpt_text = response_data["choices"][0]["message"]["content"].strip()
            print(f"ChatGPT's Response:\n{chatgpt_text}")
            return chatgpt_text
        else:
            print("ChatGPT: No content received or unexpected response structure.")
            # print(f"Full ChatGPT Response: {response_data}") # Uncomment for detailed debugging
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error communicating with ChatGPT API: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON from ChatGPT API. Response text: {response.text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred with ChatGPT: {e}")
        return None

# --- Main Interaction Logic ---
if __name__ == "__main__":
    user_initial_prompt = input("\nWhat topic or question do you want Gemini to ask ChatGPT about? (e.g., 'Jesus Blood'):\n")

    if not user_initial_prompt:
        print("No initial prompt provided. Exiting.")
    else:
        # Step 1: Gemini formulates a question for ChatGPT
        gemini_question_prompt = (
            f"Based on the user's interest in '{user_initial_prompt}', "
            "formulate a concise and clear question that I (Gemini) should ask ChatGPT. "
            "The question should be direct and suitable for a general-purpose AI. "
            "Just provide the question itself, without conversational filler."
        )
        question_for_chatgpt = ask_gemini(gemini_question_prompt)

        if question_for_chatgpt:
            # Step 2: Send Gemini's question to ChatGPT
            chatgpt_response = ask_chatgpt(question_for_chatgpt)

            if chatgpt_response:
                # Step 3: Gemini analyzes ChatGPT's response
                gemini_analysis_prompt = (
                    f"I asked ChatGPT the following question: '{question_for_chatgpt}' "
                    f"and it responded with: '{chatgpt_response}'. "
                    "Please analyze ChatGPT's response. Is it comprehensive, accurate, "
                    "and directly address the question? What are its strengths and weaknesses? "
                    "Provide a brief, objective summary of your analysis."
                )
                gemini_analysis = ask_gemini(gemini_analysis_prompt)

                if gemini_analysis:
                    print("\n--- Interaction Complete ---")
                    print("Gemini has asked ChatGPT and analyzed the response.")
                else:
                    print("\n--- Interaction Incomplete ---")
                    print("Gemini could not analyze ChatGPT's response.")
            else:
                print("\n--- Interaction Incomplete ---")
                print("ChatGPT did not provide a valid response.")
        else:
            print("\n--- Interaction Incomplete ---")
            print("Gemini could not formulate a question for ChatGPT.")